## Import Libraries

In [7]:
# %matplotlib ipympl
# %matplotlib inline
%matplotlib wx

In [8]:
import matplotlib.pyplot as plt
plt.ion()

In [9]:
from pydgilib_extra import *
from atprogram.atprogram import atprogram

In [10]:
from os import getcwd, path, pardir
import pickle

## Compile and program project

In [11]:
project_path = [path.curdir, "AES_Flash-S"]
project_path

['.', 'AES_Flash-S']

In [30]:
atprogram(path.abspath(path.join(*project_path)), verbose=2)

Building file: .././main.c
Invoking: ARM/GNU C Compiler : 6.3.1
"C:\Program Files (x86)\Atmel\Studio\7.0\toolchain\arm\arm-gnu-toolchain\bin\arm-none-eabi-gcc.exe"  -x c -mthumb -D__SAML11E16A__ -DDEBUG  -I"C:\Program Files (x86)\Atmel\Studio\7.0\Packs\arm\cmsis\5.0.1\CMSIS\Include" -I"../Config" -I".." -I"../examples" -I"../hal/include" -I"../hal/utils/include" -I"../hpl/core" -I"../hpl/crya" -I"../hpl/dmac" -I"../hpl/gclk" -I"../hpl/mclk" -I"../hpl/nvmctrl" -I"../hpl/osc32kctrl" -I"../hpl/oscctrl" -I"../hpl/pm" -I"../hpl/port" -I"../hri" -I"../trustzone" -I"../../../../shared" -I"C:\Program Files (x86)\Atmel\Studio\7.0\Packs\Atmel\SAML11_DFP\1.0.109\include"  -O1 -ffunction-sections -mlong-calls -g3 -Wall -mcpu=cortex-m23 -c -std=gnu99 -mcmse -MD -MP -MF "main.d" -MT"main.d" -MT"main.o"   -o "main.o" ".././main.c" 
Finished building: .././main.c
Building target: AES_Flash-S.elf
Invoking: ARM/GNU Linker : 6.3.1
"C:\Program Files (x86)\Atmel\Studio\7.0\toolchain\arm\arm-gnu-toolchain\b

0

## Data Logging

In [12]:
live_plot = False

Create a figure for the plot.

In [13]:
if live_plot:
    fig = plt.figure(figsize=(6, 4))
    fig.show()

Create the configuration dictionary for `DGILibExtra`.

In [14]:
config_dict = {
    "loggers": [LOGGER_OBJECT, LOGGER_CSV],
    "file_name_base": "experiment_aes_flash"
}
config_dict_plot = {
    "loggers": [LOGGER_OBJECT, LOGGER_PLOT, LOGGER_CSV],
    "plot_pins": [False, False, True, True],
    "plot_pins_method": "line",
    "plot_xmax": 5,
    "window_title": "Experiment AES-256 Flash",
}

Stop criteria to pass to the logger:

In [15]:
def stop_fn(logger_data):
    return all(logger_data.gpio.values[-1])

Perform the measurement.

In [35]:
data = []
cd = config_dict.copy()
if live_plot:
    fig.clf()
    for ax in fig.get_axes():
        ax.cla()
    
    cd.update(config_dict_plot)
    cd["fig"] = fig
    
with DGILibExtra(**cd) as dgilib:
    dgilib.device_reset()
    dgilib.logger.log(1000,stop_fn)
    data = dgilib.data

# Store Data

In [36]:
import pickle

In [36]:
pickle.dump(data, open("aes_flash_logger_data.p", "wb"))

# Load Data

In [18]:
data = pickle.load(open("aes_flash_logger_data.p", "rb"))

In [17]:
iteration = 0
name = "AES-256_Flash"
data = pickle.load(open(path.join(path.pardir, path.pardir, f"{name}_{iteration}.p"), "rb"))

## Analysis

Create Stop Function to stop parsing the data when all pins are high.

In [19]:
def stop_function(pin_values):
    return all(pin_values)

Parse the data.

In [20]:
aes_charge, aes_time = power_and_time_per_pulse(data, 2, stop_function=stop_function)

In [21]:
flash_charge, flash_time = power_and_time_per_pulse(data, 3, stop_function=stop_function)

In [22]:
length = len(aes_charge)
assert length == len(aes_time)
assert length == len(flash_charge)
assert length == len(flash_time)
print(length)

750


In [23]:
aes_encrypt_charge = aes_charge[0::2]
aes_decrypt_charge = aes_charge[1::2]
aes_encrypt_time = aes_time[0::2]
aes_decrypt_time = aes_time[1::2]

In [24]:
aes_flash_write_charge = flash_charge[0::2]
aes_flash_read_charge = flash_charge[1::2]
aes_flash_write_time = flash_time[0::2]
aes_flash_read_time = flash_time[1::2]

In [25]:
len(aes_encrypt_charge), len(aes_decrypt_charge), len(aes_encrypt_time), len(aes_decrypt_time), len(aes_flash_write_charge), len(aes_flash_read_charge), len(aes_flash_write_time), len(aes_flash_read_time)

(375, 375, 375, 375, 375, 375, 375, 375)

In [26]:
drop = 0
cutoff = min(len(aes_encrypt_charge), len(aes_decrypt_charge), len(aes_encrypt_time), len(aes_decrypt_time), len(aes_flash_write_charge), len(aes_flash_read_charge), len(aes_flash_write_time), len(aes_flash_read_time)) - drop
aes_encrypt_charge = aes_encrypt_charge[:cutoff]
aes_decrypt_charge = aes_decrypt_charge[:cutoff]
aes_encrypt_time = aes_encrypt_time[:cutoff]
aes_decrypt_time = aes_decrypt_time[:cutoff]
aes_flash_write_charge = aes_flash_write_charge[:cutoff]
aes_flash_read_charge = aes_flash_read_charge[:cutoff]
aes_flash_write_time = aes_flash_write_time[:cutoff]
aes_flash_read_time = aes_flash_read_time[:cutoff]

In [27]:
length = len(aes_encrypt_charge)
assert length == len(aes_decrypt_charge)
assert length == len(aes_encrypt_time)
assert length == len(aes_decrypt_time)
assert length == len(aes_flash_write_charge)
assert length == len(aes_flash_read_charge)
assert length == len(aes_flash_write_time)
assert length == len(aes_flash_read_time)
print(length)

375


# Convert to Joule

In [28]:
voltage = 3.33
j_scale = 1e3 # m
t_scale = 1e3 # m
model_j_scale = 1e6 # n
model_t_scale = 1e3 # u

experiment_name = 'AES-256'

In [29]:
aes_encrypt_energy = aes_encrypt_charge[:cutoff]
aes_flash_write_energy = aes_flash_write_charge[:cutoff]
aes_flash_read_energy = aes_flash_read_charge[:cutoff]
aes_decrypt_energy = aes_decrypt_charge[:cutoff]
aes_encrypt_time_s = aes_encrypt_time[:cutoff]
aes_flash_write_time_s = aes_flash_write_time[:cutoff]
aes_flash_read_time_s = aes_flash_read_time[:cutoff]
aes_decrypt_time_s = aes_decrypt_time[:cutoff]

for i in range(len(aes_encrypt_energy)):
    aes_encrypt_energy[i] = aes_encrypt_energy[i] * voltage * j_scale
for i in range(len(aes_flash_write_energy)):
    aes_flash_write_energy[i] = aes_flash_write_energy[i] * voltage * j_scale
for i in range(len(aes_flash_read_energy)):
    aes_flash_read_energy[i] = aes_flash_read_energy[i] * voltage * j_scale
for i in range(len(aes_decrypt_energy)):
    aes_decrypt_energy[i] = aes_decrypt_energy[i] * voltage * j_scale
    
for i in range(len(aes_encrypt_time_s)):
    aes_encrypt_time_s[i] = aes_encrypt_time_s[i] * t_scale
for i in range(len(aes_flash_write_time_s)):
    aes_flash_write_time_s[i] = aes_flash_write_time_s[i] * t_scale
for i in range(len(aes_flash_read_time_s)):
    aes_flash_read_time_s[i] = aes_flash_read_time_s[i] * t_scale
for i in range(len(aes_decrypt_time_s)):
    aes_decrypt_time_s[i] = aes_decrypt_time_s[i] * t_scale

In [30]:
MBEDTLS_AES_BLOCK_SIZE = 16
STEP_SIZE = MBEDTLS_AES_BLOCK_SIZE
MIN_NUM_BYTES = STEP_SIZE
num_bytes = range(MIN_NUM_BYTES, MIN_NUM_BYTES + STEP_SIZE * len(aes_encrypt_energy), STEP_SIZE)
print(f"MAX_NUM_BYTES: {num_bytes[-1]}")

MAX_NUM_BYTES: 6000


In [31]:
from lmfit import Model

def line(x, slope, intercept):
    """a line"""
    return [slope*i + intercept for i in x]

mod = Model(line)
pars = mod.make_params(slope=0, intercept=1)
# pars['intercept'].set(min=0)

In [57]:
results = []
ylabels = (['Energy [mJ]'] * 2 + ['Time [ms]'] * 2) * 2 + ['Energy [mJ]'] + ['Time [ms]']
parameter_names = [
    'Encrypt Energy',
    'Flash Write Energy',
    'Flash Read Energy',
    'Decrypt Energy',
    'Encrypt Time',
    'Flash Write Time',
    'Flash Read Time',
    'Decrypt Time',
    'Total Energy',
    'Total Time',
]
for y in [aes_encrypt_energy, aes_flash_write_energy, aes_flash_read_energy, aes_decrypt_energy, aes_encrypt_time_s, aes_flash_write_time_s, aes_flash_read_time_s, aes_decrypt_time_s,
          [e + w + r + d for (e,w,r,d) in zip(aes_encrypt_energy, aes_flash_write_energy, aes_flash_read_energy, aes_decrypt_energy)],
          [e + w + r + d for (e,w,r,d) in zip(aes_encrypt_time_s, aes_flash_write_time_s, aes_flash_read_time_s, aes_decrypt_time_s)]]:
    result = mod.fit(y, pars, x=num_bytes)
    print(result.fit_report())
    fig, grid = result.plot(
        xlabel='Checkpoint Size [Bytes]',
        ylabel=ylabels[len(results)])
    fig.tight_layout(rect=(0.05, 0.05, 1, 1))
    fig.set_size_inches(5, 4.5, forward=True)
    fig.canvas.set_window_title(
        f"Residuals of {experiment_name} {parameter_names[len(results)]}")
    fig.show()
    results.append(result)

[[Model]]
    Model(line)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 6
    # data points      = 375
    # variables        = 2
    chi-square         = 4.8262e-05
    reduced chi-square = 1.2939e-07
    Akaike info crit   = -5945.66985
    Bayesian info crit = -5937.81600
[[Variables]]
    slope:      1.0720e-04 +/- 1.0724e-08 (0.01%) (init = 0)
    intercept: -5.3293e-04 +/- 3.7225e-05 (6.98%) (init = 1)
[[Correlations]] (unreported correlations are < 0.100)
    C(slope, intercept) = -0.867

[[Model]]
    Model(line)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 6
    # data points      = 375
    # variables        = 2
    chi-square         = 0.01197788
    reduced chi-square = 3.2112e-05
    Akaike info crit   = -3877.85741
    Bayesian info crit = -3870.00356
[[Variables]]
    slope:      8.9432e-05 +/- 1.6895e-07 (0.19%) (init = 0)
    intercept:  0.01535507 +/- 5.8643e-04 (3.82%) (init = 1)
[[Correlations]] (unrepor

In [33]:
fig2 = plt.figure(figsize=(8, 6))
fig2.canvas.set_window_title(f"Analysis {experiment_name}")

In [34]:
charge_color = 'r'
time_color = 'b'

In [35]:
fig2.clf()
# fig2.suptitle("Energy analysis of AES")
ax1 = fig2.add_subplot(1, 1, 1)
ax2 = ax1.twinx()
ax1.set_xlabel('Checkpoint Size [Bytes]')
ax1.set_ylabel('Energy [mJ]', color=charge_color)
ax2.set_ylabel('Time [ms]', color=time_color)
ax1.tick_params('y', colors=charge_color)
ax2.tick_params('y', colors=time_color)

In [50]:
lines = []
lines += ax1.plot(num_bytes, aes_encrypt_energy, charge_color+'-', label=f'{parameter_names[len(lines)]}')
lines += ax1.plot(num_bytes, aes_flash_write_energy, charge_color+'-.', label=f'{parameter_names[len(lines)]}')
lines += ax1.plot(num_bytes, aes_flash_read_energy, charge_color+':', label=f'{parameter_names[len(lines)]}')
lines += ax1.plot(num_bytes, aes_decrypt_energy, charge_color+'--', label=f'{parameter_names[len(lines)]}')
lines += ax2.plot(num_bytes, aes_encrypt_time_s, time_color+'-', label=f'{parameter_names[len(lines)]}')
lines += ax2.plot(num_bytes, aes_flash_write_time_s, time_color+'-.', label=f'{parameter_names[len(lines)]}')
lines += ax2.plot(num_bytes, aes_flash_read_time_s, time_color+':', label=f'{parameter_names[len(lines)]}')
lines += ax2.plot(num_bytes, aes_decrypt_time_s, time_color+'--', label=f'{parameter_names[len(lines)]}')
ax1.legend(handles=lines)
ax1.set_title(
    f"{parameter_names[0]}: Slope {results[0].params['slope'].value * model_j_scale:.04} nJ/B, Intercept {results[0].params['intercept'].value * model_j_scale:.04} nJ\n" +
    f"{parameter_names[1]}: Slope {results[1].params['slope'].value * model_j_scale:.04} nJ/B, Intercept {results[1].params['intercept'].value * model_j_scale:.04} nJ\n" +
    f"{parameter_names[2]}: Slope {results[2].params['slope'].value * model_j_scale:.04} nJ/B, Intercept {results[2].params['intercept'].value * model_j_scale:.04} nJ\n" +
    f"{parameter_names[3]}: Slope {results[3].params['slope'].value * model_j_scale:.04} nJ/B, Intercept {results[3].params['intercept'].value * model_j_scale:.04} nJ\n" +
    f"{parameter_names[4]}: Slope {results[4].params['slope'].value * model_t_scale:.04} $\mu$s/B, Intercept {results[4].params['intercept'].value * model_t_scale:.04} $\mu$s\n" +
    f"{parameter_names[5]}: Slope {results[5].params['slope'].value * model_t_scale:.04} $\mu$s/B, Intercept {results[5].params['intercept'].value * model_t_scale:.04} $\mu$s\n" +
    f"{parameter_names[6]}: Slope {results[6].params['slope'].value * model_t_scale:.04} $\mu$s/B, Intercept {results[6].params['intercept'].value * model_t_scale:.04} $\mu$s\n" +
    f"{parameter_names[7]}: Slope {results[7].params['slope'].value * model_t_scale:.04} $\mu$s/B, Intercept {results[7].params['intercept'].value * model_t_scale:.04} $\mu$s\n" +
    f"{parameter_names[8]}: Slope {results[8].params['slope'].value * model_j_scale:.04} nJ/B, Intercept {results[8].params['intercept'].value * model_j_scale:.04} nJ\n" +
    f"{parameter_names[9]}: Slope {results[9].params['slope'].value * model_t_scale:.04} $\mu$s/B, Intercept {results[9].params['intercept'].value * model_t_scale:.04} $\mu$s\n")
fig2.tight_layout()
fig2.show()

In [59]:
print(
    f"{parameter_names[0]}: Slope {results[0].params['slope'].value * model_j_scale:.020} nJ/B, Intercept {results[0].params['intercept'].value * model_j_scale:.020} nJ\n" +
    f"{parameter_names[1]}: Slope {results[1].params['slope'].value * model_j_scale:.020} nJ/B, Intercept {results[1].params['intercept'].value * model_j_scale:.020} nJ\n" +
    f"{parameter_names[2]}: Slope {results[2].params['slope'].value * model_j_scale:.020} nJ/B, Intercept {results[2].params['intercept'].value * model_j_scale:.020} nJ\n" +
    f"{parameter_names[3]}: Slope {results[3].params['slope'].value * model_j_scale:.020} nJ/B, Intercept {results[3].params['intercept'].value * model_j_scale:.020} nJ\n" +
    f"{parameter_names[4]}: Slope {results[4].params['slope'].value * model_t_scale:.020} $\mu$s/B, Intercept {results[4].params['intercept'].value * model_t_scale:.020} $\mu$s\n" +
    f"{parameter_names[5]}: Slope {results[5].params['slope'].value * model_t_scale:.020} $\mu$s/B, Intercept {results[5].params['intercept'].value * model_t_scale:.020} $\mu$s\n" +
    f"{parameter_names[6]}: Slope {results[6].params['slope'].value * model_t_scale:.020} $\mu$s/B, Intercept {results[6].params['intercept'].value * model_t_scale:.020} $\mu$s\n" +
    f"{parameter_names[7]}: Slope {results[7].params['slope'].value * model_t_scale:.020} $\mu$s/B, Intercept {results[7].params['intercept'].value * model_t_scale:.020} $\mu$s\n" +
    f"{parameter_names[8]}: Slope {results[8].params['slope'].value * model_j_scale:.020} nJ/B, Intercept {results[8].params['intercept'].value * model_j_scale:.020} nJ\n" +
    f"{parameter_names[9]}: Slope {results[9].params['slope'].value * model_t_scale:.020} $\mu$s/B, Intercept {results[9].params['intercept'].value * model_t_scale:.020} $\mu$s\n"
)

Encrypt Energy: Slope 107.20141183908533833 nJ/B, Intercept -532.93235653406929941 nJ
Flash Write Energy: Slope 89.432015318286602223 nJ/B, Intercept 15355.067682061548112 nJ
Flash Read Energy: Slope 2.5849720630519326114 nJ/B, Intercept -14.22819186988029827 nJ
Decrypt Energy: Slope 111.83201687384840284 nJ/B, Intercept -545.28747116782551529 nJ
Encrypt Time: Slope 61.339934713978017555 $\mu$s/B, Intercept 16.352446417268552636 $\mu$s
Flash Write Time: Slope 14.296131583214407357 $\mu$s/B, Intercept 2234.7549069986694121 $\mu$s
Flash Read Time: Slope 2.1363268658369252151 $\mu$s/B, Intercept 12.827784387678292433 $\mu$s
Decrypt Time: Slope 64.482258911372625221 $\mu$s/B, Intercept 12.699757136522526579 $\mu$s
Total Energy: Slope 311.05041658406918259 nJ/B, Intercept 14262.617678520451591 nJ
Total Time: Slope 142.2546519574131878 $\mu$s/B, Intercept 2276.6353298404524139 $\mu$s



In [51]:
# Save Charge amount list into pickle file
import pickle
pickle.dump(aes_encrypt_energy, open("aes_flash_encrypt_energy_mJ.p", "wb"))
pickle.dump(aes_decrypt_energy, open("aes_flash_decrypt_energy_mJ.p", "wb"))
pickle.dump(aes_flash_write_energy, open("aes_flash_write_energy_mJ.p", "wb"))
pickle.dump(aes_flash_read_energy, open("aes_flash_read_energy_mJ.p", "wb"))
pickle.dump(aes_encrypt_time_s, open("aes_flash_encrypt_time_ms.p", "wb"))
pickle.dump(aes_decrypt_time_s, open("aes_flash_decrypt_time_ms.p", "wb"))
pickle.dump(aes_flash_write_time_s, open("aes_flash_write_time_ms.p", "wb"))
pickle.dump(aes_flash_read_time_s, open("aes_flash_read_time_ms.p", "wb"))

In [41]:
aes = [aes_encrypt_energy, aes_flash_write_energy, aes_flash_read_energy, aes_decrypt_energy, aes_encrypt_time_s, aes_flash_write_time_s, aes_flash_read_time_s, aes_decrypt_time_s]
for i in aes:
    print(len(i), len(i)*16)

375 6000
375 6000
375 6000
375 6000
375 6000
375 6000
375 6000
375 6000


## Write config file

In [52]:
import json

config = {}
config["name"] = "AES-256 Flash"
config["project_paths"] = [project_path]
config["config_dict"] = config_dict
config["config_dict_plot"] = config_dict_plot
config["analysis"] = {"pins":{2: ["AES-256 Encrypt", "AES-256 Decrypt"], 3: ["AES-256 Flash Write", "AES-256 Flash Read"]}, 
                      "result_types": ["Charge", "Time"],
                      "section_types": {"init": [], 
                                        "store": ["AES-256 Encrypt", "AES-256 Flash Write"],
                                        "load": ["AES-256 Flash Read", "AES-256 Decrypt"],
                                        "exit": []},
                      "labels": {
                          "Charge": {"x":"Data Size", "x_unit": "byte", "y": "Charge", "y_unit": "C"},
                          "Time": {"x":"Data Size", "x_unit": "byte", "y": "Time", "y_unit": "s"},
                      },
                      "x_step": MBEDTLS_AES_BLOCK_SIZE}

with open("looped_experiment.json", 'w') as config_file:  
    json.dump(config, config_file, indent=4)

# Write model data

In [53]:
dump_pickle = True
fit_lm = True
verbose = 2
show_lm_plot = 2

In [54]:
# Parse data
analysis_config = config.get("analysis")
result_types = analysis_config.get("result_types")
x_step = analysis_config.get("x_step")
parsed_data = {}
for pin, parameter_names in analysis_config.get("pins").items():
    data2 = power_and_time_per_pulse(
        data, int(pin), stop_function=stop_function)
    num_names = len(parameter_names)
    for i, parameter_name in enumerate(parameter_names):
        end_index = -drop * num_names or None
        parsed_data[parameter_name] = {
            result_types[0]: data2[0][i:end_index:num_names],
            result_types[1]: data2[1][i:end_index:num_names],
            "x_step": x_step}
if dump_pickle:
    pickle.dump(parsed_data, open(
        path.join(path.curdir,
                  f"{config_dict.get('file_name_base')}_looped.p"), "wb"))

In [55]:
# Fit lm
if fit_lm:
    model = None
    if model is None:
        def line(x, intercept, slope):
            """a line"""
            return [intercept + slope*i for i in x]

        model = Model(line)
        params = model.make_params(intercept=0, slope=1)
#         params['intercept'].set(min=0)
    else:
        params = model.params

    model_results = {}
    labels = analysis_config.get("labels")
    for parameter_name in parsed_data.keys():
        length = len(parsed_data[parameter_name][result_types[0]])
        x_step = parsed_data[parameter_name]["x_step"]
        num_bytes = range(x_step, (length+1)*x_step, x_step)
        if verbose:
            print(
                f"Fitting model to {parameter_name} with {length} " +
                f"samples, from {min(num_bytes)} to {max(num_bytes)} "
                f"bytes in steps of {x_step}.")
        model_result = {}
        for result_type in result_types:
            model_result[result_type] = model.fit(
                parsed_data[parameter_name][result_type], params,
                x=num_bytes)
            if verbose >= 2:
                print(model_result[result_type].fit_report())
            # Plot multiple view
            if show_lm_plot >= 2:
                fig, grid = model_result[result_type].plot(
                    xlabel=f"{labels[result_type]['x']} " +
                           f"[{labels[result_type]['x_unit']}]",
                    ylabel=f"{labels[result_type]['y']} " +
                           f"[{labels[result_type]['y_unit']}]")
                fig.canvas.set_window_title(
                    f"Residuals of {parameter_name}")
                fig.tight_layout()
                fig.show()
            model_results[parameter_name] = model_result

    # Plot single view
    if show_lm_plot:
        import matplotlib.pyplot as plt
        fig = plt.figure(figsize=(9, 6))
        fig.canvas.set_window_title(f"Analysis {config.get('name')}")
        colors = dict(zip(result_types, ['r', 'b']))
        line_styles = (
            line_style for line_style in ('-', '--', '-.', ':') * 2)
        # fig.suptitle(f"Energy analysis of {config.get('name')}")
        ax = {}
        ax[result_types[0]] = fig.add_subplot(1, 1, 1)
        ax[result_types[1]] = ax[result_types[0]].twinx()
        ax[result_types[0]].set_xlabel(
            f"{labels[result_types[0]]['x']} " +
            f"[{labels[result_types[0]]['x_unit']}]")
        for result_type in result_types:
            ax[result_type].set_ylabel(
                f"{labels[result_type]['y']} " +
                f"[{labels[result_type]['y_unit']}]",
                color=colors[result_type])
            ax[result_type].tick_params('y', colors=colors[result_type])

        lines = []
        title_str = ""
        for parameter_name in parsed_data.keys():
            length = len(parsed_data[parameter_name][result_types[0]])
            x_step = parsed_data[parameter_name]["x_step"]
            num_bytes = range(x_step, (length+1)*x_step, x_step)
            model_result = {}
            line_style = next(line_styles)
            for result_type in result_types:
                label = f"{parameter_name} {labels[result_type]['y']}"
                lines += ax[result_type].plot(
                    num_bytes, parsed_data[parameter_name][result_type],
                    colors[result_type] + line_style, label=label)
                title_str += f"{label} "
                for param in params.keys():
                    title_str += "".join(
                        f"{params[param].name.capitalize()}: ")
                    title_str += "".join(
                        f"{model_results[parameter_name][result_type].params[param].value: .03} ")
                    title_str += "".join(
                        f"{labels[result_type]['y_unit']}, ")
                title_str = title_str[:-2] + \
                    f" per {labels[result_type]['x_unit']}\n"
        ax[result_types[0]].legend(handles=lines)
        ax[result_types[0]].set_title(title_str[:-1])
        # fig.tight_layout()
        fig.tight_layout(rect=(0.05, 0.05, 1, 1))
        fig.set_size_inches(8, 6, forward=True)
        fig.show()

    # Save model results to file
    if dump_pickle:
        model_results_dump = {}
        for parameter_name in model_results.keys():
            model_results_dump[parameter_name] = {}
            for result_type in model_results[parameter_name].keys():
                model_results_dump[parameter_name][result_type] = \
                    model_results[parameter_name][result_type].values

        pickle.dump(model_results_dump, open(path.join(
            path.curdir,
            f"{config_dict.get('file_name_base')}_model.p"), "wb"))

Fitting model to AES-256 Encrypt with 375 samples, from 16 to 6000 bytes in steps of 16.
[[Model]]
    Model(line)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 10
    # data points      = 375
    # variables        = 2
    chi-square         = 4.3523e-12
    reduced chi-square = 1.1668e-14
    Akaike info crit   = -12028.7155
    Bayesian info crit = -12020.8617
[[Variables]]
    intercept: -1.6004e-07 +/- 1.1179e-08 (6.98%) (init = 0)
    slope:      3.2193e-08 +/- 3.2206e-12 (0.01%) (init = 1)
[[Correlations]] (unreported correlations are < 0.100)
    C(intercept, slope) = -0.867

[[Model]]
    Model(line)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 7
    # data points      = 375
    # variables        = 2
    chi-square         = 4.9591e-07
    reduced chi-square = 1.3295e-09
    Akaike info crit   = -7662.42239
    Bayesian info crit = -7654.56854
[[Variables]]
    intercept:  1.6355e-05 +/- 3.7735e-06 (23.07%) (init 